In [36]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import os
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder

In [4]:
#loading the dataset

train_data_init = pd.read_json('../input/stanford-covid-vaccine/train.json',lines=True)
test_data_init = pd.read_json('../input/stanford-covid-vaccine/test.json',lines=True)
sub_file = pd.read_csv('../input/stanford-covid-vaccine/sample_submission.csv')

In [3]:
test_data_init.head()

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,..."


In [5]:
train_data = train_data_init.copy()
test_data = test_data_init.copy()
#train_data = train_data[train_data['SN_filter'] > 0].reset_index(drop = True)
train_data.shape

(2400, 19)

In [23]:
test_data.head()

seq_length seq_scored sequence structure predicted_loop_type
id           step                                                             
id_00073f8be 0           107         68        G         .                   E
             1           107         68        G         .                   E
             2           107         68        A         .                   E
             3           107         68        A         .                   E
             4           107         68        A         .                   E

In [6]:
train_data_json = [json.loads(line) for line in open('../input/stanford-covid-vaccine/train.json', 'r')]
for json in train_data_json:
    json['step'] = list(range(json['seq_scored']))
    json['sequence'] = list(json['sequence'])
    json['structure'] = list(json['structure'])
    json['predicted_loop_type'] = list(json['predicted_loop_type'])
    json['unpaired_probability'] = list(1-sum(np.load('../input/stanford-covid-vaccine/bpps/'+json['id']+'.npy'))) 

In [7]:
train_data = pd.json_normalize(data = train_data_json, 
                            record_path ='reactivity',  
                            meta =['id','signal_to_noise',
                                  'SN_filter','seq_length','seq_scored']) 
train_data.rename(columns={0:'reactivity'}, inplace=True)

In [8]:
train_data['step'] = pd.json_normalize(data = train_data_json, 
                            record_path ='step'
                                        )
train_data['unpaired_probability'] = pd.json_normalize(data = train_data_json, 
                            record_path ='unpaired_probability'
                                        )
train_data['sequence'] = pd.json_normalize(data = train_data_json, 
                            record_path ='sequence'
                                        )
train_data['structure'] = pd.json_normalize(data = train_data_json, 
                            record_path ='structure'
                                        )
train_data['predicted_loop_type'] = pd.json_normalize(data = train_data_json, 
                            record_path ='predicted_loop_type'
                                        )
train_data['reactivity_error'] = pd.json_normalize(data = train_data_json, 
                            record_path ='reactivity_error'
                                        )
train_data['deg_Mg_pH10'] = pd.json_normalize(data = train_data_json, 
                            record_path ='deg_Mg_pH10'
                                        )
train_data['deg_error_Mg_pH10'] = pd.json_normalize(data = train_data_json, 
                            record_path ='deg_error_Mg_pH10'
                                        )
train_data['deg_pH10'] = pd.json_normalize(data = train_data_json, 
                            record_path ='deg_pH10',
                                        )
train_data['deg_error_pH10'] = pd.json_normalize(data = train_data_json, 
                            record_path ='deg_error_pH10',
                                        )
train_data['deg_Mg_50C'] = pd.json_normalize(data = train_data_json, 
                            record_path ='deg_Mg_50C',
                                        )
train_data['deg_error_Mg_50C'] = pd.json_normalize(data = train_data_json, 
                            record_path ='deg_error_Mg_50C',
                                        )
train_data['deg_50C'] = pd.json_normalize(data = train_data_json, 
                            record_path ='deg_50C',
                                        )
train_data['deg_error_50C'] = pd.json_normalize(data = train_data_json, 
                            record_path ='deg_error_50C',
                                        )

train_data.set_index(['id','step'], inplace=True)
train_data = train_data[train_data['SN_filter'] > 0].reset_index(drop = False)
#train_data.set_index(['id','step'], inplace=True)

In [9]:
import json
from pandas.io.json import json_normalize
import os
test_data_json = [json.loads(line) for line in open('../input/stanford-covid-vaccine/test.json', 'r')]

In [10]:
for json in test_data_json:
    json['step'] = list(range(json['seq_scored']))
    json['sequence'] = list(json['sequence'])
    json['structure'] = list(json['structure'])
    json['predicted_loop_type'] = list(json['predicted_loop_type'])

test_data = pd.json_normalize(data = test_data_json, 
                            record_path ='step',  
                            meta =['id','seq_length','seq_scored']) 
test_data.rename(columns={0:'step'}, inplace=True)

test_data['sequence'] = pd.json_normalize(data = test_data_json, 
                            record_path ='sequence'
                                        )
test_data['structure'] = pd.json_normalize(data = test_data_json, 
                            record_path ='structure'
                                        )
test_data['predicted_loop_type'] = pd.json_normalize(data = test_data_json, 
                            record_path ='predicted_loop_type')

test_data.set_index(['id','step'], inplace=True)

In [10]:
test_data.shape

(316227, 5)

In [11]:
test_data.head()

seq_length seq_scored sequence structure predicted_loop_type
id           step                                                             
id_00073f8be 0           107         68        G         .                   E
             1           107         68        G         .                   E
             2           107         68        A         .                   E
             3           107         68        A         .                   E
             4           107         68        A         .                   E

In [11]:
# Apply one-hot encoder to each column with categorical data
cat_col = ['sequence','structure','predicted_loop_type']
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
train_data_OH = pd.DataFrame(OH_encoder.fit_transform(train_data[cat_col]))
train_data_OH.columns = OH_encoder.get_feature_names(cat_col)
# One-hot encoding removed index; put it back
train_data_OH.index = train_data.index
# Remove categorical columns (will replace with one-hot encoding)
train_data_num = train_data.drop(cat_col, axis=1)
# Add one-hot encoded columns to numerical features
train_data_OH = pd.concat([train_data_num, train_data_OH], axis=1)
train_data_OH.head()

,id,step,reactivity,signal_to_noise,SN_filter,seq_length,seq_scored,unpaired_probability,reactivity_error,deg_Mg_pH10,...,structure_(,structure_),structure_.,predicted_loop_type_B,predicted_loop_type_E,predicted_loop_type_H,predicted_loop_type_I,predicted_loop_type_M,predicted_loop_type_S,predicted_loop_type_X
0,id_001f94081,0,0.3297,6.894,1,107,68,0.801458,0.1359,0.7556,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,id_001f94081,1,1.5693,6.894,1,107,68,0.816288,0.2070,2.9830,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,id_001f94081,2,1.1227,6.894,1,107,68,0.939998,0.1633,0.2526,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,id_001f94081,3,0.8686,6.894,1,107,68,0.986878,0.1452,1.3789,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,id_001f94081,4,0.7217,6.894,1,107,68,0.988720,0.1314,0.6376,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [23]:
X=train_data_OH.drop(['id', 'step', 'unpaired_probability','reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C', 
                 'deg_error_50C', 'SN_filter', 'signal_to_noise', 'deg_pH10', 'deg_50C','reactivity',
                      'deg_Mg_pH10', 'deg_Mg_50C'], axis=1)
y=train_data_OH[['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']]

In [24]:
X.head()
#y.head()

,seq_length,seq_scored,sequence_A,sequence_C,sequence_G,sequence_U,structure_(,structure_),structure_.,predicted_loop_type_B,predicted_loop_type_E,predicted_loop_type_H,predicted_loop_type_I,predicted_loop_type_M,predicted_loop_type_S,predicted_loop_type_X
0,107,68,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,107,68,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,107,68,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,107,68,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,107,68,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [25]:
from sklearn.model_selection import train_test_split
# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size=0.20)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((86441, 16), (21611, 16), (86441, 3), (21611, 3))

In [26]:
#Calculates competition eval metric
from sklearn.metrics import make_scorer
def mcrmse_loss(y_true, y_pred, N=3):
    assert len(y_true) == len(y_pred)
    n = len(y_true)
    return np.sum(np.sqrt(np.sum((y_true - y_pred)**2, axis=0)/n)) / N

custom_scorer = make_scorer(mcrmse_loss, greater_is_better=False)

In [27]:
#Standardize the data sets
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)

In [28]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
# Train using parameters
xgb = XGBRegressor(
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1,
    reg_alpha=1,
    n_estimators=800,
    learning_rate=0.1,
    random_state=42
)

multi_op_reg = MultiOutputRegressor(xgb)
multi_op_reg.fit(X_train, y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=0.1,
                                            max_delta_step=None, max_depth=5,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=800, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, reg_alpha=1,
                                            reg_

In [29]:
# Train score
mcrmse_loss(np.array(y_train), multi_op_reg.predict(X_train))

0.48890695338218887

In [30]:
# Validation score
mcrmse_loss(np.array(y_valid), multi_op_reg.predict(X_valid))

0.48604808679768535

In [31]:
from sklearn.preprocessing import OneHotEncoder
# Apply one-hot encoder to each column with categorical data
cat_col1 = ['sequence','structure','predicted_loop_type']
OH_encoder1 = OneHotEncoder(handle_unknown='ignore', sparse=False)
test_data_OH = pd.DataFrame(OH_encoder1.fit_transform(test_data[cat_col1]))
test_data_OH.columns = OH_encoder1.get_feature_names(cat_col1)
# One-hot encoding removed index; put it back
test_data_OH.index = test_data.index
# Remove categorical columns (will replace with one-hot encoding)
test_data_num = test_data.drop(cat_col1, axis=1)
# Add one-hot encoded columns to numerical features
test_data_OH = pd.concat([test_data_num, test_data_OH], axis=1)
test_data_OH.head()

seq_length seq_scored  sequence_A  sequence_C  sequence_G  \
id           step                                                             
id_00073f8be 0           107         68         0.0         0.0         1.0   
             1           107         68         0.0         0.0         1.0   
             2           107         68         1.0         0.0         0.0   
             3           107         68         1.0         0.0         0.0   
             4           107         68         1.0         0.0         0.0   

                   sequence_U  structure_(  structure_)  structure_.  \
id           step                                                      
id_00073f8be 0            0.0          0.0          0.0          1.0   
             1            0.0          0.0          0.0          1.0   
             2            0.0          0.0          0.0          1.0   
             3            0.0          0.0          0.0          1.0   
             4            0.0          0.0          0.0          1.0   

                   predicted_loop_type_B  predicted_loop_type_E  \
id           step                                                 
id_00073f8be 0                       0.0                    1.0   
             1                       0.0                    1.0   
             2                       0.0                    1.0   
             3                       0.0                    1.0   
             4                       0.0                    1.0   

                   predicted_loop_type_H  predicted_loop_type_I  \
id           step                                                 
id_00073f8be 0                       0.0                    0.0   
             1                       0.0                    0.0   
             2                       0.0                    0.0   
             3                       0.0                    0.0   
             4                       0.0                    0.0   

                   predicted_loop_type_M  predicted_loop_type_S  \
id           step                                                 
id_00073f8be 0                       0.0                    0.0   
             1                       0.0                    0.0   
             2                       0.0                    0.0   
             3                       0.0                    0.0   
             4                       0.0                    0.0   

                   predicted_loop_type_X  
id           step                         
id_00073f8be 0                       0.0  
             1                       0.0  
             2                       0.0  
             3                       0.0  
             4                       0.0

In [21]:
test_data_OH.shape

(316227, 16)

In [32]:
test_data_OH = scaler.transform(test_data_OH)

In [42]:
# Predict
preds = pd.DataFrame(multi_op_reg.predict(test_data_OH))

In [46]:
preds.shape

(316227, 3)

In [40]:
def build_id_seqpos(df):
    id_seqpos = []
    for i in range(len(df)):
        id = df.loc[i, 'id']
        seq_length = df.loc[i, 'seq_length']
        for seqpos in range(seq_length):
            id_seqpos.append(id + '_' + str(seqpos))
    return id_seqpos

id_seqpos = build_id_seqpos(test_data_init)

In [47]:
df_sub = pd.DataFrame({'id_seqpos': id_seqpos})
df_sub.shape

ValueError: array length 457953 does not match index length 316227